In [50]:
import torch
import opt_einsum as oe
import numpy as np

In [59]:
import torch

r = torch.randn

v1 = r(1,3)
v2 = r(1000,3)
m1 = r(1,3,3)
m2 = r(1000,3,3)
t1 = r(1,3,3,3)
t2 = r(1000,3,3,3)

In [60]:
oe.contract_path('bjkl, bmno, bmj, bo, bn, bkl -> b', t2, t2, m2, v2, v2, m2)

([(0, 5), (1, 4), (1, 2), (0, 2), (0, 1)],
   Complete contraction:  bjkl,bmno,bmj,bo,bn,bkl->b
          Naive scaling:  7
      Optimized scaling:  4
       Naive FLOP count:  4.374e+6
   Optimized FLOP count:  1.410e+5
    Theoretical speedup:  3.102e+1
   Largest intermediate:  9.000e+3 elements
 --------------------------------------------------------------------------------
 scaling        BLAS                current                             remaining
 --------------------------------------------------------------------------------
    4              0           bkl,bjkl->bj                  bmno,bmj,bo,bn,bj->b
    3              0             bj,bmj->bm                      bmno,bo,bn,bm->b
    3              0             bn,bo->bno                        bmno,bm,bno->b
    4              0           bno,bmno->bm                              bm,bm->b
    2              0               bm,bm->b                                  b->b)

In [61]:
expr = oe.contract_expression('bjkl, bmno, bmj, bo, bn, bkl -> b', t2.shape, t2.shape, m2.shape, v2.shape, v2.shape, m2.shape)

expr

<ContractExpression('bjkl, bmno, bmj, bo, bn, bkl -> b')>

In [75]:
no_saved_path = %timeit -o -q -n 1000 torch.einsum('bjkl, bmno, bmj, bo, bn, bkl -> b', t2, t2, m2, v2, v2, m2)
no_saved_path_np = %timeit -o -q -n 1000 np.einsum('bjkl, bmno, bmj, bo, bn, bkl -> b', t2.numpy(), t2.numpy(), m2.numpy(), v2.numpy(), v2.numpy(), m2.numpy())
saved_path = %timeit -o -q -n 1000 expr(t2, t2, m2, v2, v2, m2)

In [80]:
no_saved_path_np.average / no_saved_pa2th.average, no_saved_path_np.average / saved_path.average

(5.3213238375998975, 27.677688078191792)

In [77]:
no_saved_path, no_saved_path_np, saved_path

(<TimeitResult : 1.79 ms ± 33.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)>,
 <TimeitResult : 9.52 ms ± 124 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)>,
 <TimeitResult : 344 µs ± 20.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)>)